In [1]:
using AdaStress
import AdaStress.BlackBox

┌ Info: Precompiling AdaStress [f8632b6a-8763-4da0-bfaf-5f7707adef25]
└ @ Base loading.jl:1317


## Simulation

In [2]:
using Distributions

In [3]:
Base.@kwdef mutable struct Walk1D <: AdaStress.BlackBox
    t::Float64=0.0
    t_max::Float64=10.0
    x::Float64=0.0
    x_fail::Float64=10.0
    dist::Normal=Normal(0.0, 1.0)
end

function initialize!(sim::Walk1D)
    sim.t = 0.0
    sim.x = 0.0
end

function update!(sim::Walk1D)
    sim.t += 1.0
    a = rand(sim.dist)
    sim.x += a
    return logpdf(sim.dist, a) - logpdf(sim.dist, 0.0)
end

update! (generic function with 1 method)

In [4]:
AdaStress.reset!(sim::Walk1D) = initialize!(sim)
AdaStress.step!(sim::Walk1D) = update!(sim)
AdaStress.isterminal(sim::Walk1D) = sim.t >= sim.t_max
AdaStress.isevent(sim::Walk1D) = sim.x >= sim.x_fail
AdaStress.distance(sim::Walk1D) = max(sim.x_fail - sim.x, 0.0)

## Solver

In [5]:
mcs = AdaStress.Solvers.MCS(num_steps=100_000)

AdaStress.Solvers.MonteCarloSearch.MCS(100000, 10)

In [6]:
mdp = AdaStress.ASTMDP(Walk1D())
mdp.reward.event_bonus = 100.0

100.0

## Replay

In [7]:
using Random

In [8]:
Random.seed!(0)
sol = mcs(() -> mdp)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


DataStructures.PriorityQueue{Any, Any, Base.Order.ForwardOrdering} with 10 entries:
  UInt32[0xd8f490c5, 0xadadd09f, 0x915650eb, 0x89477a2d, 0xd7aa5f01,… => 3.82247
  UInt32[0xc266b27f, 0xd90a54f4, 0x2147d790, 0xecf6ab21, 0x12836c2b,… => 4.00441
  UInt32[0xf4f3e560, 0xf0c4e711, 0xaddae546, 0xd7c9cee2, 0xd11d3f31,… => 99.8553
  UInt32[0x1537a859, 0xa698790b, 0xb4882912, 0xeaa1eb27, 0xcc73de6e,… => 100.94
  UInt32[0x41774728, 0xcb252bc1, 0x5ae29330, 0x3b84c57e, 0xd563571d,… => 101.003
  UInt32[0xeeea0e31, 0xcbbff7b8, 0x64079b5d, 0xe11a0405, 0xb149e5b4,… => 101.343
  UInt32[0x7b83b232, 0x043d5ba0, 0xc9ec8f89, 0x2d5fe6a4, 0x874b8d46,… => 101.603
  UInt32[0xad514d56, 0x45ee1802, 0xce6189c3, 0x44a59f45, 0x36e679e9,… => 101.787
  UInt32[0x71952c2b, 0xefbb016c, 0x33766911, 0xa9f38c69, 0xb7548e27,… => 102.152
  UInt32[0x97ce4d7d, 0x0c4bd4e9, 0x7f749b76, 0xc755abf3, 0xb43cd020,… => 102.525

In [9]:
best_path = collect(keys(sol))[end]
AdaStress.reset!(mdp.sim)
for seed in best_path
    (mdp)(seed)
end

In [10]:
mdp.sim

Walk1D(10.0, 10.0, 10.259967025330106, 10.0, Normal{Float64}(μ=0.0, σ=1.0))